In [1]:
## Importation des bibliothèques (utilisation de l'alias standard pd)
import pandas as pds
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import tree
import statistics

In [2]:
## Lecture
df = pds.read_csv('./train.csv', sep=",") # Utilisation de df comme nom principal

FileNotFoundError: [Errno 2] No such file or directory: './train.csv'

In [84]:
# Suppression des colonnes (identifiants et colonnes complexes/inutiles)
df = df.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)

# Suppression des lignes où Embarked ou Fare sont manquants (selon votre choix)
df = df.dropna(subset=['Embarked', 'Fare'])

In [85]:
# Encodage binaire de 'Sex' (Label Encoding)
df['Sex'] = df['Sex'].replace(['male','female'],[0,1])

C:\Users\louis\AppData\Local\Temp\ipykernel_24956\633968186.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sex'] = df['Sex'].replace(['male','female'],[0,1])


In [86]:
df_encoded_all = pds.get_dummies(df, columns=['Embarked'], drop_first=True)


In [87]:
df_encoded_all

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,0,22.0,1,0,7.2500,False,True
1,1,1,1,38.0,1,0,71.2833,False,False
2,1,3,1,26.0,0,0,7.9250,False,True
3,1,1,1,35.0,1,0,53.1000,False,True
4,0,3,0,35.0,0,0,8.0500,False,True
...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,False,True
887,1,1,1,19.0,0,0,30.0000,False,True
888,0,3,1,NaN,1,2,23.4500,False,True
889,1,1,0,26.0,0,0,30.0000,False,False


In [88]:
# --- 1. Séparation des données pour l'imputation ---

# Features de clustering (toutes les colonnes sauf Age et Survived)
features = [col for col in df_encoded_all.columns if col not in ['Age', 'Survived']]

X_full = df_encoded_all[features]
y_age = df_encoded_all['Age']

# Séparation des lignes à connaître (Age non-NaN) et à imputer (Age NaN)
X_age_connu = X_full[y_age.notnull()]
X_age_manquant = X_full[y_age.isnull()]

# --- 2. Mise à l'échelle des Features de Clustering (CRUCIAL!) ---

scaler = StandardScaler()
X_scaled_connu = scaler.fit_transform(X_age_connu)
X_scaled_manquant = scaler.transform(X_age_manquant)

In [89]:
df_encoded = df_encoded.dropna()

In [90]:
# Les features pour le clustering
X_train_cluster = df_encoded.loc[df_age_connu.index]
y_age_train = df_age_connu['Age']

# Mise à l'échelle (CRUCIAL pour K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train_cluster)

# 1. Définir le nombre de clusters K. (Il faudrait optimiser ce K, mais disons K=5 pour l'exemple)
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)

# 2. Entraîner le modèle sur les données avec Age connu
kmeans.fit(X_scaled)

# 3. Ajouter l'étiquette de cluster au DataFrame
df_age_connu['Cluster'] = kmeans.labels_
# Calculer l'âge moyen pour chaque cluster
moyenne_age_par_cluster = df_age_connu.groupby('Cluster')['Age'].mean()
print("Moyennes d'âge par cluster :")
print(moyenne_age_par_cluster)

c:\Users\louis\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


Moyennes d'âge par cluster :
Cluster
0    21.234909
1    27.984694
2    26.831897
3    31.600000
4     9.600000
5    32.078400
6    27.685185
7    39.538462
8    35.872123
9    37.071429
Name: Age, dtype: float64


C:\Users\louis\AppData\Local\Temp\ipykernel_24956\2881048221.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_connu['Cluster'] = kmeans.labels_


In [94]:
# --- 1. Imputation de l'Âge ---
K_optimal = 10 # <--- REMPLACEZ PAR VOTRE VALEUR OPTIMALE

# Entraînement final
kmeans_final = KMeans(n_clusters=K_optimal, random_state=42, n_init=10)
kmeans_final.fit(X_scaled_connu)

# Créer un DataFrame avec Age et Cluster pour calculer les moyennes
df_age_connu_labels = pds.DataFrame(X_scaled_connu, index=X_age_connu.index)
df_age_connu_labels['Age'] = y_age.dropna()
df_age_connu_labels['Cluster'] = kmeans_final.labels_

# Calculer la moyenne d'âge par cluster
moyenne_age_par_cluster = df_age_connu_labels.groupby('Cluster')['Age'].mean()

# Prédire le cluster pour les NaN
clusters_manquants = kmeans_final.predict(X_scaled_manquant)

# Imputation
ages_imputes = [moyenne_age_par_cluster[cluster] for cluster in clusters_manquants]
imputation_serie = pds.Series(ages_imputes, index=X_age_manquant.index)

# Mettre à jour la colonne 'Age' dans le DataFrame principal
df_encoded_all.loc[df_encoded_all['Age'].isnull(), 'Age'] = imputation_serie

print(f"Imputation de l'âge par clustering terminée. NaN restants dans Age : {df_encoded_all['Age'].isnull().sum()}")


c:\Users\louis\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


Imputation de l'âge par clustering terminée. NaN restants dans Age : 0


In [95]:
df = df_encoded_all

In [96]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,0,22.000000,1,0,7.2500,False,True
1,1,1,1,38.000000,1,0,71.2833,False,False
2,1,3,1,26.000000,0,0,7.9250,False,True
3,1,1,1,35.000000,1,0,53.1000,False,True
4,0,3,0,35.000000,0,0,8.0500,False,True
...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.000000,0,0,13.0000,False,True
887,1,1,1,19.000000,0,0,30.0000,False,True
888,0,3,1,25.947073,1,2,23.4500,False,True
889,1,1,0,26.000000,0,0,30.0000,False,False
